# We Rate Dogs. Wrangling and analysis project

First, we're getting the basics out of the way...


In [1]:
#Basics
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import datetime
%matplotlib inline


# Gather files and resources as needed for the project.
This means loading the files supplied and downloading the image predictions.
The resources we need to get are:
- The archive 'twitter-archive-enhanced.csv' from udacity. Downloaded and added to the project
- The image predictions files which will be programatically downloaded
- info for all the tweets, accessed from through the twitter api


In [2]:
#Files

#Read the csv file and have a look at it.

archive = pd.read_csv('twitter-archive-enhanced.csv')
archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [3]:
#Download the tsv file for image predictions
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open(url.split('/')[-1], mode='wb') as file:
    file.write(response.content)
    
img_pred = pd.read_csv('image-predictions.tsv', sep='\t')
img_pred.head()


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [4]:
#Twitter Stuff - Api access

import tweepy

consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
#Testing that Twitter works
tweet = api.get_status(archive.tweet_id[1], tweet_mode='extended')
info = tweet._json
info

TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]

In [ ]:
#Get all the tweets
#Saving out the errors in a txt file, just in case

#set this in order to grab or not grab the data from twitter
getData = 0
if(getData==1):
    data = []
    problem_tweets = {}
    tweetcount =archive.tweet_id.size
    #tweetcount = 40 #This one is just for troubleshooting

    for a in range(0, tweetcount):
        currentid = archive.tweet_id[a]
        print("requesting tweet id: {} at {}".format(currentid, datetime.datetime.now().time()))
        try:
            tweet = api.get_status(archive.tweet_id[a], tweet_mode='extended')
            data.append(tweet._json)
        except Exception as e:
            print("failed tweet id: {} at {}".format(currentid, datetime.datetime.now().time()))    
            problem_tweets[currentid] = str(e)

    with open("tweet_json.txt", mode='w') as file:
        json.dump(data, file)
    
    #Saving the errors, just in case we need them at some point...
    f = open("tweet_errors.txt","w")
    f.write(str(problem_tweets))
    f.close()

    print("done")
    print("issues with tweets: {}".format(problem_tweets))
elif(getData==0):
    print("Not grabbing data, assuming you already have it.")


# Turn the twitter data into something we can use
I want to store these fields:
- tweet id (required)
- retweet count (required)
- favorite count (required)
- followers count (already formulating ideas that it could be interesting to see if followe counts have a relative impact on favorites and retweets)

In [ ]:
#Turn json into dataframe.

#Load and parse the file
tweet_list = []
with open('tweet_json.txt') as file:
    alltweets = json.load(file)
    for tweet in alltweets:
        tweet_list.append({'tweet_id' : tweet['id'], 'retweet_count' : tweet['retweet_count'],
                          'favorite_count': tweet['favorite_count'], 'followers_count' : tweet['user']['followers_count']})

#Turn the file into a dataframe
tw_api_data = pd.DataFrame(tweet_list, columns = ['tweet_id', 'retweet_count', 'favorite_count',
                                                 'followers_count'])

#Quickly have a look to see that all is going according to plan
tw_api_data.info()

# Data gathering roundup

We now have three data sources in dataframes. 
- _tw_api_data_ holds the data we extrated from the twitter API.
- _archive_ The data uploaded, from the file supplied by udacity. 
- _img_pred_ The predictions from the tab seperated file downloaded programatically.



# Assessing the data prior to cleaning
Now that all the sources are in place, I want to look at them and see what needs cleaning, tidying and wrangling.

I will start with the Archive... There are things I know we need to pay attention to:
- Dog name
- Stages
- Rating

In [ ]:
# Starting from the top. The Archive

print(archive.info())
archive.head()

In [ ]:
# A quick check to see how a row looks

print(archive.iloc[0])

It seems we don't need the source column, and we can see that the link is embedded in the text field as well. This quick look also made me interested to see if expanded_urls sometimes hold more than one url.

In [ ]:
archive.expanded_urls.str.split(',')

In [ ]:
# Lets see how many names have been captured

archive['name'].value_counts()

This introduces quite a few things we need to look at already, lets start the list:
745 names are missing
8 dogs called the, and 55 dogs called a

So we need to find those names, and I suspect that filtering for lowercase names will give me a list of dogs that need special attention as well. I will have a quick look at this straight away.

In [ ]:
lowerFrame = archive[archive['name'].str[0].str.islower()]
lowerFrame['name'].value_counts()

Ok, so I will make the assumption that nobody calls their dog 'quite' or 'this' ... though 'Unaccaptable' and 'Mad' could be funny names. But in all seriousness, we need to address these.

# Dog stages
Next up we look into the stages of the dogs.

In [ ]:
(archive.loc[:,'doggo':'puppo'] != 'None').sum()

In [ ]:
#Out of curiosity I want to see if there are dogs that have been classified with more than one type
multidogs = 0
unclasdogs = 0
singleclasdogs = 0
for index, row in archive.iterrows():
    dogtypecount = 0
    if row['doggo'] != 'None':
        dogtypecount += 1
    if row['floofer'] != 'None':
        dogtypecount += 1
    if row['pupper'] != 'None':
        dogtypecount += 1
    if row['puppo'] != 'None':
        dogtypecount += 1
    if dogtypecount >=2:
        multidogs += 1
    elif dogtypecount ==1:
        singleclasdogs += 1
    else:
        unclasdogs += 1

print("there are {} dogs with more than one classification".format( multidogs))
print("there are {} dogs with just  one classification".format( singleclasdogs))
print("there are {} dogs with no classification".format( unclasdogs))

Ok, so there are already a lot of things to look at, but let us continue the assesment.

# Dog ratings

time to asses the ratings. These are the most important features, if the name of the account is to be taken seriously.


In [ ]:
archive['rating_numerator'].value_counts()


In [ ]:
archive['rating_denominator'].value_counts()


allright.... it's all over the place. There is no missing data it seems, but people seem liberal in keeping to actual ratings that would make sense. But we could likely find interesting things from this at some point. 

The last thing I want to look at is to check if we have any retweets or replies in there that we should get rid of. 

# No retweets or replies!



In [ ]:
#need to see a bit more
pd.set_option('max_colwidth',280)

#lets look at the text in some of the replies
archive[archive.in_reply_to_user_id.notnull()].text

In [ ]:
#and the same for the tweets that have been retweeted
archive[archive.retweeted_status_user_id .notnull()].text



From reading through quite a few of both outputs, I will deem these rows unusable.

# Quality and tidiness issues to fix on the Archive dataframe:

Quality:
- remove rows that are replies or retweets.
- Work on classification of dogs
- Remove names with lowercase start
- Find dog names where missing and possible
- Clean links from text


Tidiness:
- I want to clean the categorization. Having the row name and variable be the same is redundant and annoying
- Once we have removed the retweets and replies, we can get rid of those columns.

Now let's move on to the images and see what other things we can find...

# img_pred



In [ ]:
#Getting an overview...
img_pred.sample(20)


Allright, there are some things we need to look into here. 

Ofcourse we need to merge this into the archive data, but we need to clean some things first.
- it doesnt seem like we need the img_num column
- There are quite a few predictions that are not dogs. we need to see how many have non-dog predictions in all three predictions. 
- We should only have on prediction per entry so probably the highest rated one would make sense.
- A quick sanity check that we do not have missing fields etc.

In [ ]:
# first things first...
img_pred.info()

The only thing that springs to mind is that we only have 2075 entries. This needs to be dealt with.

In [ ]:
#Lets see how many entries to not have a dog in any prediction.
notDog = 0
for index, row in img_pred.iterrows():
    if ((int(row['p1_dog'])+int(row['p2_dog'])+int(row['p3_dog'])) == 0):
        notDog += 1

print(notDog)

Ok, so that seems pretty straight forward. We do need to choose a way to move forward with regards to the dogs images that are not classified as dogs. Looking through some of them, it seems the prediction is actually right in many cases, so it might make sense to get rid of those. Another option is to just not have anything in that field.

# Quality and tidiness issues to fix on the img_pred dataframe:

Quality:
- keep just the highest confidence prediction that is also a dog. (let's keep the confidence as well)

Tidiness:
- get rid of the img_num column
- fix upper and lower cases

Other things:
- once merged, we need to see what the issue is with missing predictions.

# Twitter api download

Time to move on to asses the newly downloaded data. Again, this one is mostly sanity checking, so let's get to it

In [ ]:
tw_api_data.info()

In [ ]:
#Lets have a look at some histograms to see if we can learn more...

tw_api_data.hist(bins=40,figsize=(16,10))

Nothing too surprising here. The most interesting thing as I see it, but this gets into analysis, is that it seems there is a very 'all or nothing' distribution to observe, though this is a pretty normal long tail. 
I am actually happy with this dataframe, I just need to merge it into the other data.

# Cleaning the data! 

It's now time to start cleaning. The issues I want to deal with are:

_Archive_
- remove rows that are retweets.
- remove rows that are replies.
- remove columns related to replies and retweets
- Dog names. I need to clean the ones that have been wrongly extracted. And try to extract more.
- Classification. Just one per dog. And try to extract more where possible
- Classifaction will go in one column
- remove urls from text.

_image predictions_
- remove predictions for non-dogs
- have just one prediction with a confidence level per dog
- remove img_num column
- fix upper and lower cases in prediction names


Next up we need to merge everything into one nice, big dataframe and have a look at that. I know there will be an issue of some entries missing that we need to look into. So let's get started.

# Wrangling the Archive

__Define__: 

1- Drop duplicates, if there are any, as a first step. This way we avoid working on more data than we need. Then create a new dataframe that we can keep working with.

__Code__;

In [ ]:
# Starting with the simplest part of cleaning in order to not spend time on things that will be removed.

print(archive.duplicated().sum())
archive_new = archive.drop_duplicates()



__Test__:

In [ ]:
archive_new.duplicated().sum()

__Define__: 

2- Remove rows that are retweets. These have values in the retweeted_status_user_id column, so using that as an identifier

__Code__;

In [ ]:
#drop rows that are retweets
archive_new = archive_new[archive_new.retweeted_status_user_id.isnull()]



__Test__:

In [ ]:
#lets check it
archive_new.info()

__Define__: 

3- Remove rows that are replies. These have values in the reply_to_user_id column, so using that as an identifier

__Code__;

In [ ]:
#drop rows that are replies
archive_new = archive_new[archive_new.in_reply_to_user_id.isnull()]


__Test__:

In [ ]:
#lets check it
archive_new.info()

__Define__: 

4- Get rid of the now empty columns for replies and rewteets.

__Code__;

In [ ]:
#Get rid of the empty columns and source column while we are at it.

archive_new.drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id',
                 'retweeted_status_user_id','retweeted_status_timestamp','source'], axis=1, inplace=True)



__Test__:

In [ ]:
archive_new.info()

__Define__: 

5- Remove the names if they start with a lower case letter

__Code__;

In [ ]:
# First, we need to remove all the lowercase names, as we found earlier that these were not names.

for index, row in archive_new.iterrows():
    if row['name'][0].islower():
        print(index)
        archive_new.drop(index, inplace=True)



__Test__:

In [ ]:
lowerStart = 0
for index, row in archive_new.iterrows():
    if row['name'][0].islower():
        lowerStart += 1

print("There are {} names that start with a lowercase letter".format(lowerStart))

__Define__: 

6- Change the names that are ste to None to actual blanks. This will make it easer to search and gather an overview etc.

__Code__;

In [ ]:
# change the names that are None to actual blanks
for index, row in archive_new.iterrows():
    if row['name'] == "None":
        archive_new.loc[index, 'name'] = None

__Test__:

In [ ]:
archive_new.name.value_counts()

In [ ]:
pd.set_option('max_colwidth',180) #Making things easier to see

#And just to make sure that worked and have a look at the text fields

archive_new[archive_new.name.isnull()].text.sample(25)

Unfortunately it doesnt seem like these have names that can be extracted automatically, so I guess we need to have a lot of dogs without names. At least we've taken out many of the errornurous names.

__Define__:

7- Remove links from the text fields to simplify and avoid whatever things could cause errors later on when extracting data from the field.

__Code__:

In [ ]:
#Removing links from text fields
for index, row in archive_new.iterrows():
    archive_new.loc[index, 'text'] = row['text'].rsplit('http', 1)[0].rsplit('http', 1)[0]

archive_new.sample(20)

Allright, that takes care of that. Now I 'just' need to wrangle the stages. I just want to make sure we have one column, and that has the name of the stage in it, and is empty if not identified. Since a floofer can also be a catch-all for just about any dog, that has lowest priority in case there are more than one stage noted for a tweet. Next comes Puppo, then pupper and lastly doggo. Mainly because the description describes them as inhereting traits for from the former stage.

__Define__:

8- Clean up dog stages. The current configuration is not making life easer for us, so the stages will go into one single column, instead of four, and in the process we are getting rid of multiple classifications per dog.

__Code__:

In [ ]:
archive_new['dog_stage'] = None
for index, row in archive_new.iterrows():
    outStage = ""
    if row['floofer'] != 'None':
        outStage = 'Floofer'
    if row['puppo'] != 'None':
        outStage = 'Puppo'
    if row['pupper'] != 'None':
        outStage = 'Pupper'
    if row['doggo'] != 'None':
        outStage = 'Doggo'
    if outStage != "":
        archive_new.loc[index, 'dog_stage'] = outStage



__Test__:

In [ ]:
archive_new.sample(40)

__Define__:

9- Drop the old columns that had the dog stages in them,.

__Code__:

In [ ]:
pd.set_option('max_colwidth',40) #Making things easier to see

#That seemed to work! Time to get rid of the old columns

archive_new.drop(['floofer', 'puppo', 'pupper','doggo'], axis=1, inplace=True)


__Test__:

In [ ]:
archive_new.head(15)

In [ ]:
#Cleaning up a bit
archive_clean = archive_new.copy()


Ok, there is ofcourse many more things to do, but for now we have dealt with:

- Removing names that were not names
- Removing links from text field
- Remove retweets
- Remove Replies
- Remove columns for retweets and replies
- Convert the four stages to a single column
- Change names that were 'None' to actual None's
- Remove now useless columns stating the stages
- Copied to archive_clean


# Img_pred wrangling

image predictions

my plan here is to deal with these issues:
- remove predictions for non-dogs
- have just one prediction with a confidence level per dog
- remove img_num column
- fix upper and lower cases in prediction names to all lower case, just for conformity



In [ ]:
img_pred_clean = img_pred.copy()
img_pred_clean.sample(8)

__Define__:

10a- Clean up the predictions by keeping just the highest rated prediction that is also a dog. If none of the predictions are dogs, the predition will be set to None with a confidence of 0. 
10b- While running through the data we will convert to lowercase.

__Code__:

In [ ]:
#Create a row for the dog race and confidence
img_pred_clean['dog_type'] = None
img_pred_clean['type_confidence'] = 0.0
#Now, we grab the highest confidence prediction, that is a dog. 

#This is 10a
for index, row in img_pred_clean.iterrows():
    outPred = ""
    outConf = 0.0
    if row['p1_dog'] == True:
        outPred = row['p1']
        outConf = row['p1_conf']
    elif row['p2_dog'] == True:
        outPred = row['p2']
        outConf = row['p2_conf']
    elif row['p3_dog'] == True:
        outPred = row['p3']
        outConf = row['p3_conf']
    else:
        outPred = None
        outConf = 0.0
    
    #Before Storing convert to lower case if not None
    #This is 10b
    if outPred:
        outPred = outPred.lower()
    img_pred_clean.loc[index, 'dog_type'] = outPred
    img_pred_clean.loc[index, 'type_confidence'] = outConf



__Test__:

In [ ]:
img_pred_clean.sample(20)

__Define__:

11- drop columns from the old predictions

__Code__:

In [ ]:
#Remove columns for predictions
img_pred_clean.drop(['p1','p1_conf', 'p1_dog','p2','p2_conf', 'p2_dog','p3','p3_conf', 'p3_dog'], axis=1, inplace=True)



__Test__:

In [ ]:
img_pred_clean.head()

__Define__:

12- Remove the img_num colum

__Code__:

In [ ]:
#and lets drop the img_num column as well
img_pred_clean.drop(['img_num'], axis=1, inplace=True)


__Test__:

In [ ]:
img_pred_clean.tail()

In [ ]:
img_pred_clean.dog_type.value_counts()

# Let the merging begin! 


In [ ]:
#img_pred_clean.head()
#archive_clean.info()

archive_merge = archive_clean.merge(img_pred_clean , on='tweet_id')
archive_merge.sample(20)

In [ ]:
#And lets add the data from the twitter api

archive_merge = archive_merge.merge(tw_api_data , on='tweet_id')
archive_merge.sample(20)

In [ ]:
#Just in case I need it, I'll calculate the rating
archive_merge['rating'] = archive_merge['rating_numerator']/archive_merge['rating_denominator']
archive_merge.head()

So even though there is still plenty of things to do and doublecheck, I am satisfied with the dataset I have now. So it's time to move on to...

# Analysis

First of i'll store the new dataset so it's easier to work with from here, and i'll load that back into a new dataframe.

Then I'll take a look at the data and see what insights I can get from it, and document and visualize these

In [ ]:
archive_merge.to_csv('twitter_archive_master.csv', index = None, header=True)
data_clean = pd.read_csv('twitter_archive_master.csv')
data_clean.info()

In [ ]:
data_clean.hist(figsize=(28,14), bins=40)

In [ ]:
#Lets see what happens when we plot the rating againts favourited.

plt.figure(figsize =(20,10))
plt.scatter((data_clean['rating']), data_clean['favorite_count'].transform(lambda x: np.log10(x)),s=10)
plt.title('rating vs favourites')
plt.xlabel('Rating')
plt.ylabel('favourited')
plt.xlim(-1,3) #in dialing this in, I had to take the outliers out of the picture to see what was going on

plt.savefig('rate_v_favor.png')
plt.show()
data_clean.corr(method='pearson')

There does not seem to be a clear correlation but it does seem that you get more favourites from ratings over 1.0 and that once the rating gets to 1.5 or higher, favouriting ceases. This can be obeserved in the graph from the clustering but does not appear in the correlation matrix.

_An observation_

It seems that nothing in the dataframe correlates linearly, which actually makes sense, as there are no rules for how ratings are given per se, so it is very much a case of controlled randomness.

In [ ]:
#Maybe looking at dog stages can lead to something interesting?

stage_rate = data_clean.groupby(['dog_stage','rating'])
stage_rate.count()

There could be something there... let's keep looking at the bigger picture first though.

In [ ]:
pd.plotting.scatter_matrix(data_clean, figsize=(20,15))

While there seems to be a few things that jump out, I cannot use the rating column against the original ratings, since it's derrived from there. Also, the tweet_id and type_confidence can't really say anything of value here.

In [ ]:
## Maybe looking over time can help us learn something
from matplotlib.dates import DateFormatter


#Time format: 2017-07-30 15:58:51 +0000

# Define the date format
myFmt = DateFormatter('%Y-%m-%d') 

data_clean['short_timestamp'] = data_clean['timestamp'].astype('datetime64[ns]')

# plot the data
fig, (ax, ax2) = plt.subplots(2, figsize=(15, 9))

ax.plot(data_clean.short_timestamp, data_clean.favorite_count.rolling(90, win_type='triang').sum())
ax2.plot(data_clean.short_timestamp, data_clean.rating.rolling(90, win_type='triang').sum())

ax.set_title('Development of favourite counts over time')
ax.set(xlabel="Time", ylabel="Favourite Counts",)
ax.xaxis_date()
ax.xaxis.set_major_formatter(myFmt)

ax2.set_title('Ratingss over time')
ax2.set(xlabel="Time", ylabel="Rating",)
ax2.xaxis_date()
ax2.xaxis.set_major_formatter(myFmt)

fig.savefig('rate_v_favor_over_time.png')



It seems that while favourites increase over time, so does the ratings of dogs. The two major spikes seen in the ratings can also be seen in the favourites. 

In [ ]:
#data_clean[data_clean.dog_stage == 'Pupper']['rating'].hist()
fig, ([ax, ax2],[ax3,ax4]) = plt.subplots(nrows=2, ncols=2, figsize=(18, 9))

ax.hist(data_clean[data_clean.dog_stage == 'Pupper']['rating'])
ax2.hist(data_clean[data_clean.dog_stage == 'Puppo']['rating'])
ax3.hist(data_clean[data_clean.dog_stage == 'Doggo']['rating'])
ax4.hist(data_clean[data_clean.dog_stage == 'Floofer']['rating'])
ax.set_title('Pupper')
ax2.set_title('Puppo')
ax3.set_title('Doggo')
ax4.set_title('Floofer')
ax.set(xlabel="Rating", ylabel="Counts",)
ax2.set(xlabel="Rating", ylabel="Counts",)
ax3.set(xlabel="Rating", ylabel="Counts",)
ax4.set(xlabel="Rating", ylabel="Counts",)

fig.savefig('rate_v_dogStage.png')


ok, from this little graph we can see a few interesting things as well. First off,if we take the outliers in Puppers out of the picture, all the ratings seem to be left-skewed. That means that people tend to rate their dogs higher when measured agaist the average. We can also see that owners of Puppers are more likely to actually rate their dogs in an 'expected' manner, with a greater deal of ratings not going over 1.0, though it's not something they adhere to in general. 
The insights gained is that people tend to rate their dogs not just above average (at least in the sense that an average rating should fall withing 0-1.0) but slightly above 1, leaning towards higher ratings. Interestingly enough, the dog owners are somewhat concious enough to not overwhelmingly rate their dogs at much more than 1. 